## Load Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy import sparse
import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.layers import Dense, Dropout,Input
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import categorical_accuracy #AUC 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,SGD
from spektral.layers import GINConv,GCNConv #, GCSConv, GlobalAvgPool
from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.data import DisjointLoader, BatchLoader, Dataset, Graph
from spektral.transforms.normalize_adj import NormalizeAdj
from spektral.transforms.normalize_one import NormalizeOne
from spektral.transforms.normalize_sphere import NormalizeSphere
import gc
import spektral.datasets
from spektral.data import DisjointLoader, BatchLoader
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from spektral.models.gcn import GCN 
from spektral.models.general_gnn import GeneralGNN
import glob

# Create Dataset Class

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy import sparse
import gc
from spektral.data import Dataset, Graph

# Custom dataset class
class MyDataset(Dataset):

    def __init__(self, graph_feature_files, ncol_files, **kwargs):
        # Store the graph feature and ncol file lists
        self.graph_feature_files = graph_feature_files
        self.ncol_files = ncol_files
        super().__init__(**kwargs)

    def download(self):
        pass

    def read(self):
        # We must return a list of Graph objects
        output = []

        # Iterate through graph_feature_files and ncol_files
        for graph_feature_file, ncol_file in zip(self.graph_feature_files, self.ncol_files):
            # Read graph features
            x_tmp = pd.read_csv(graph_feature_file, sep=",", header=0)
            # Replace background for normal
            x_tmp['label'] = x_tmp['label'].replace('background','normal')

           
            # Read graph topology
            a_tmp = pd.read_csv(ncol_file, sep=" ", header=None, names=["source", "target", "weight"])

            # Replace all non-zero values in the 'weight' column with 1
            a_tmp.loc[a_tmp['weight'] != 0, 'weight'] = 1
            
            # Create dictionaries that identify each node and label with an integer
            class_idx = {name: idx for idx, name in enumerate(sorted(x_tmp["label"].unique()))}
            node_idx = {name: idx for idx, name in enumerate(sorted(x_tmp["node"].unique()))}

            # Change node names and label for their corresponding integer
            x_tmp["node"] = x_tmp["node"].apply(lambda name: node_idx[name])
            x_tmp["label"] = x_tmp["label"].apply(lambda value: class_idx[value])
            a_tmp["source"] = a_tmp["source"].apply(lambda name: node_idx[name])
            a_tmp["target"] = a_tmp["target"].apply(lambda name: node_idx[name])

            # Node features: all but node and label
            x = x_tmp.sort_values("node")[x_tmp.columns.difference(["node","label"], sort=False)]
            # normalize Node features
            x = (x - x.min()) / (x.max() - x.min())
            # Convert to numpy
            x = x.to_numpy()
            x = x.astype(np.float32)

            # Create adjacency matrix from source, target, and weight
            a_source = a_tmp[["source"]].to_numpy().T
            a_source = np.reshape(a_source, a_source.shape[-1])
            a_target = a_tmp[["target"]].to_numpy().T
            a_target = np.reshape(a_target, a_target.shape[-1])
            a_weight = a_tmp[["weight"]].to_numpy().T
            a_weight = np.reshape(a_weight, a_weight.shape[-1])

            # Adjacency matrix:
            #a = sparse.coo_matrix((a_weight, (a_source, a_target)), shape=(x.shape[0], x.shape[0]))
            a = sparse.csr_matrix((a_weight, (a_source, a_target)), shape=(x.shape[0], x.shape[0]), dtype=np.float32)

            # Label (CTU13random):
            #y = []
            #for j in range(x_tmp.shape[0]):
            #     np.random.seed(200+j)
            #     y_tmp = np.random.randint(0,2)
            #     if y_tmp:
            #         y.append(np.array([1., 0.]))
            #     else:
            #         y.append(np.array([0., 1.]))
            #y = np.array(y)
            #y_one_hot = y.astype(np.int64)

            #y = []
            #for j in range(x_tmp.shape[0]):
            #    #if((x[j,0] > 1) or (x[j,1] > 1)): # Condition for CTU13balancedIDOD: 
            #    if (x_tmp.iloc[j,4] > 2):  
            #        y.append(np.array([0., 1.])) # clase 1 = "infected"
            #    else:
            #        y.append(np.array([1., 0.])) # clase 0 = "normal"
            #y = np.array(y)
            #y_one_hot = y.astype(np.float32)



            # Label:
            y = x_tmp.sort_values("node")["label"].to_numpy()
            y.astype(np.int64)
            y_one_hot = np.eye(2)[y]
            
            # Create a Graph object and add it to the output list
            output.append(Graph(x=x, a=a, y=y_one_hot))

            # Free memory
            del x_tmp, x, a_tmp, a_source, a_target, a_weight, a, y
            gc.collect()

        return output



## Instance CTU13 Dataset

In [ ]:
# Create a list of graph feature files and ncol files
graph_feature_files = sorted(glob.glob("/mnt/CEPH/ctu13/features/*"))
ncol_files = sorted(glob.glob("/mnt/CEPH/ctu13/ncol/*"))
# Instantiate the dataset
dataset = MyDataset(graph_feature_files, ncol_files,transforms=[NormalizeAdj(symmetric=False)])
#dataset = MyDataset(graph_feature_files, ncol_files)

In [ ]:
for j in range(13):
    suma = 0
    for i in range(dataset[j].n_nodes):
        if all(dataset[j].y[i] == [0.,1.]):
            suma+=1
    print("cap ",j+1,": infected:",suma, " - normal:",dataset[j].n_nodes-suma, " -- prop:",suma/dataset[j].n_nodes)

## Split train/test

In [ ]:
# Remove capture 9 from dataset
dataset_test2 = dataset[9]
dataset = dataset[:8] + dataset[9:]

split = int(0.8 * len(dataset))
dataset_train, dataset_test = dataset[:split], dataset[split:]

In [ ]:
dataset_train
dataset_test[0].n_nodes
dataset_train.n_labels

## Loaders

In [ ]:
batch_size = 1
loader_train = DisjointLoader(dataset_train, node_level= True, batch_size=batch_size, epochs=300, shuffle=False, )
loader_test = DisjointLoader(dataset_test, node_level = True, batch_size=batch_size)

# Create a custom GCN model (not used)

In [ ]:
def create_gcn_model():
    # Define input placeholders for node features, adjacency matrix, and segment indices
    X_in = Input(shape=(dataset.n_node_features,))
    A_in = Input((None,), sparse=True)
    I_in = Input(shape=(), dtype=tf.int32)

    # Apply the first GINConv layer with 32 units and ReLU activation
    X_1 = GINConv(32, activation="relu")([X_in, A_in])
    # Apply dropout with a rate of 0.5
    X_1 = Dropout(0.5)(X_1)

    # Apply the second GINConv layer with 32 units and ReLU activation
    X_2 = GINConv(32, activation="relu")([X_1, A_in])
    # Apply dropout with a rate of 0.5
    X_2 = Dropout(0.5)(X_2)

    # Aggregate the node features using the segment_mean function and the segment indices
    X_3 = tf.math.segment_mean(X_2, I_in)
    # Apply a dense output layer with the number of labels and softmax activation
    out = Dense(dataset.n_labels, activation="softmax")(X_3)

    # Create and return the model with the defined inputs and outputs
    model = Model(inputs=[X_in, A_in, I_in], outputs=out)
    return model

# Current implementation of GCN

## Implementation of focal loss

In [ ]:
import tensorflow as tf
from tensorflow.keras.losses import Loss

class FocalLoss(Loss):
    def __init__(self, alpha=0.25, gamma=2.0, name="focal_loss"):
        super().__init__(name=name)
        self.alpha = alpha
        self.gamma = gamma

    def call(self, y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-8, 1 - 1e-8)
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        loss = -self.alpha * (1 - pt) ** self.gamma * tf.math.log(pt)
        return tf.reduce_sum(loss, axis=-1)


class BinaryFocalLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=0.25, gamma=2.0, from_logits=False, **kwargs):
        super().__init__(**kwargs)
        # Initialize the hyperparameters alpha and gamma
        self.alpha = alpha
        self.gamma = gamma
        # If True, the input to the loss function is assumed to be logits
        self.from_logits = from_logits

    def call(self, y_true, y_pred):
        # If from_logits is True, apply sigmoid activation to logits (y_pred)
        if self.from_logits:
            y_pred = tf.nn.sigmoid(y_pred)

        # Cast y_true to float32
        y_true = tf.cast(y_true, dtype=tf.float32)

        # Clip y_pred values between epsilon and (1 - epsilon) to avoid log(0)
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)

        # Compute binary cross-entropy
        cross_entropy = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)
        
        # Compute focal loss
        focal_loss = self.alpha * tf.pow(1 - y_pred, self.gamma) * cross_entropy

        # Reduce focal_loss along the last axis and then calculate the mean over the batch
        return tf.reduce_mean(tf.reduce_sum(focal_loss, axis=-1))

    def get_config(self):
        # Get the configuration of the loss function, including alpha, gamma, and from_logits
        config = super().get_config()
        config.update({"alpha": self.alpha, "gamma": self.gamma, "from_logits": self.from_logits})
        return config



## Load GCN model and parameters

In [ ]:
#model = create_gcn_model()
model = GCN(n_labels=dataset.n_labels)
optimizer = Adam(learning_rate=0.0001)
#optimizer = SGD(learning_rate =0.0001)
#loss_fn = BinaryCrossentropy(from_logits=False)
#loss_fn = CategoricalCrossentropy(from_logits=False)
loss_fn= BinaryFocalLoss(alpha=0.25, gamma=2.0)
auc_metric = AUC()

## Function for training the model

In [ ]:
# Decorate the function with @tf.function to compile as a TensorFlow graph
# Use the input_signature from loader_train and relax shapes for varying graph sizes

#class_weights = np.array([0.0000001, 0.9999999])

@tf.function(input_signature=loader_train.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    #print(inputs.shape)
    # Create a GradientTape context to record operations for automatic differentiation
    with tf.GradientTape() as tape:
        # Compute model predictions with the inputs, set training=True for training-specific behaviors
        predictions = model(inputs, training=True)

        ### CLASS WEIGHT
        ## Create a tensor with class weights
        #sample_weights = tf.gather(class_weights, tf.argmax(target, axis=-1))
        # Calculate the loss using the provided loss_fn and add the model's regularization losses and including sample_weight
        #loss = loss_fn(target, predictions,sample_weight=sample_weights) + sum(model.losses)

        
        # Calculate the loss using the provided loss_fn and add the model's regularization losses
        loss = loss_fn(target, predictions) + sum(model.losses)


    # Compute gradients of the loss with respect to the model's trainable variables
    gradients = tape.gradient(loss, model.trainable_variables)
    # Apply the gradients to the model's variables using the optimizer's apply_gradients method
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Compute the accuracy using the categorical_accuracy function from TensorFlow
    # Calculate the mean accuracy using tf.reduce_mean
    acc = tf.reduce_mean(categorical_accuracy(target, predictions))
     # Update the AUC metric
    auc_metric.update_state(target, predictions)
    # Get the current AUC value
    auc = auc_metric.result()


    # Return the loss and accuracy as output
    return loss, acc

### Debug dataset loaders

In [ ]:
print("Loader input signature:", loader_train.tf_signature())
sample_batch = loader_train.__next__()
inputs, targets = sample_batch
for tensor in inputs:
    print(tensor.shape, tensor.dtype)
print("Targets shape:", targets.shape, "dtype:", targets.dtype)

## Function to evaluate the performance of the model

In [ ]:

def evaluate_auc(loader):
    output = []
    step = 0
    auc_metric_test = AUC()

    while step < loader.steps_per_epoch:
        step += 1
        inputs, target = loader.__next__()
        pred = model(inputs, training=False)
        
        # Update the AUC metric with the true labels and predictions
        auc_metric_test.update_state(target, pred)
        
        outs = (
            loss_fn(target, pred),
            auc_metric_test.result().numpy(),  # Get the current AUC value
            len(target),  # Keep track of batch size
        )
        output.append(outs)
        if step == loader.steps_per_epoch:
            output = np.array(output)
            return np.average(output[:, :-1], 0, weights=output[:, -1])


In [ ]:
def evaluate(loader):
    output = []
    step = 0
    while step < loader.steps_per_epoch:
        step += 1
        inputs, target = loader.__next__()
        pred = model(inputs, training=False)
        outs = (
            loss_fn(target, pred),
            tf.reduce_mean(categorical_accuracy(target, pred)),
            len(target),  # Keep track of batch size
        )
        output.append(outs)
        if step == loader.steps_per_epoch:
            output = np.array(output)
            return np.average(output[:, :-1], 0, weights=output[:, -1])

# MAIN CODE for training

In [ ]:
# Initialize the epoch and step counters to -1
# Create an empty list for storing training results
epoch = step = -1
results = []
results_train = []
# Iterate through the batches in the loader_train data loader
for batch in loader_train:
    inputs,targets = batch
    targets = tf.convert_to_tensor(targets)
    # Increment the step counter
    step += 1

    # Execute the train_step function with the current batch
    # Obtain the loss and accuracy
    loss, metric = train_step(inputs,targets)

    # Append the loss and accuracy to the results list
    results.append((loss, metric))
    results_train.append( np.mean(results,0) )
    # Check if the current step is equal to the number of steps per epoch (loader_train.steps_per_epoch)
    if step == loader_train.steps_per_epoch:
        # Reset the step counter to 0
        # Increment the epoch counter
        step = 0
        epoch += 1

        # Evaluate the model on the test set using the evaluate function (which should be defined beforehand)
        # Store the test results in results_te
        results_te = evaluate(loader_test)

        # Print the epoch number, mean training loss and accuracy, and test loss and accuracy
        print(
            "Ep. {} - Loss: {:.3f} - Metric: {:.3f} - Test loss: {:.3f} - Test Metric: {:.3f}".format(
                epoch, *np.mean(results, 0), *results_te
            )
        )

        # Reset the results list to start collecting results for the next epoch
        results = []

## Plot results

In [ ]:
import matplotlib.pyplot as plt
losses_train = [item[0] for item in results_train]
auc_train = [item[1] for item in results_train]

# Create a list of epoch numbers
epochs = list(range(1, len(losses_train) + 1))

# Create a new figure and axis
fig, ax1 = plt.subplots()

# Plot losses_train on the first y-axis
ax1.plot(epochs, losses_train, 'b-', label='Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color='b')
ax1.tick_params('y', colors='b')

# Create a second y-axis to plot auc_train
ax2 = ax1.twinx()
ax2.plot(epochs, auc_train, 'r-', label='ACC')
ax2.set_ylabel('ACC', color='r')
ax2.tick_params('y', colors='r')

# Add legends for both y-axes
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Show the plot
plt.show()


In [ ]:
sample_batch = loader_test.__next__()
inputs, targets = sample_batch
for tensor in inputs:
    print(tensor.shape, tensor.dtype)
print("Targets shape:", targets.shape, "dtype:", targets.dtype)


# Evaluate Predictions

In [ ]:

predictions = []
targets = []
step = 0
while step < loader_test.steps_per_epoch:
        step += 1
        inputs, target = loader_test.__next__()
        pred = model(inputs, training=False)
        predictions.append(pred)
        targets.append(target)
    
predictions = np.vstack(predictions)
    
targets = np.vstack(targets)
# Post-process the predictions if necessary (e.g., convert probabilities to class labels)
predicted_values = np.argmax(predictions, axis=1)
true_labels = np.argmax(targets,axis =1 )


In [ ]:

from sklearn.metrics import confusion_matrix,roc_auc_score,accuracy_score

conf_mat = confusion_matrix(predicted_values, true_labels)
print(conf_mat)
#


In [ ]:
auc_score = roc_auc_score(true_labels, predicted_values)
accuracy_score = accuracy_score(true_labels, predicted_values)

print("AUC Score:", auc_score)
print("Accuracy Score:", accuracy_score)

In [ ]:
accuracy_score